# 矯正 Meme Generator Data Set 的 URL

由於 [Meme Generator Data Set](https://www.kaggle.com/electron0zero/memegenerator-dataset/home) 的 URL 無法直接連到圖片連結，故設計此 Code 處理之：

1. 載入套件

In [1]:
import numpy as np
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import re

2. 撰寫從 http://webarchive.loc.gov 取得第一筆備份連結的函數

In [2]:
def get_arc_link(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    return soup.find("a", href=re.compile("/h"))['href']

3. 設定起始參數

In [3]:
startRows = 1
nRowsRead = 10
totalRaw = 57652

4. 逐行替換

In [4]:
curRows = 180
while curRows < totalRaw:
    df = pd.read_csv('memegenerator.csv', delimiter=',',
                      nrows = nRowsRead, skiprows = range(1,curRows)) 
    # 處理無法讀取圖片的連結
    for i in range(0, len(df)):
        new_url = get_arc_link(df["Archived URL"][i])
        df.loc[i, "Archived URL"] = new_url
    df
    # 儲存整理過後 csv
    if(curRows == startRows):
        df.iloc[:,:-1].to_csv('memegenerator_arrange.csv',
                              index=0)
    else:
        df.iloc[:,:-1].to_csv('memegenerator_arrange.csv', mode='a',
                              index=0, header=False)

    # 決定下次讀取行數    
    curRows += nRowsRead
    if((curRows + nRowsRead) > totalRaw):
        nRowsRead = totalRaw - curRows + 1

KeyboardInterrupt: 